# 微博平台社交行为预测分析
对于一条原创博文而言，转发，评论，点赞等互动行为能够体现出用户对于博文内容的兴趣程度，也是对博文进行分发控制的重要参考指标。本届赛题的任务就是根据抽样用户的原创博文在发表一天之后的转发，评论，赞总数，并预测用户后续博文在发表一天之后的互动情况。

微博的社交互动行为应该取决于三种因素：
1. 发布微博的用户特征，比如发表微博的平均社交次数，活跃时间等等
2. 微博中的形式化特征，比如发布时间，表情标签，@标签，#标签等
3. 微博中的非形式特征，比如语言风格，主题等等

由于原始数据基本上没有任何的特征，所以先做特征工程处理，然后再进行探索性数据分析，并基于这些分析结果来构造出更多的候选特征。

### 加载历史数据集

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import re

train_data_path = 'data/weibo_train_data.txt'
test_data_path = 'data/weibo_predict_data.txt'

def read_history_data(data_path, columns):
    temp_data_list = []
    with open(data_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            temp_data_list.append(line.strip('\n').split('\t'))
    return pd.DataFrame(temp_data_list, columns=columns)

train_dataframe = read_history_data(train_data_path, columns=['用户id', '博文id', '时刻', '转发数量', '评论数量', '点赞数量', '博文内容'])
test_dataframe = read_history_data(test_data_path, columns=['用户id', '博文id', '时刻', '博文内容'])

训练集的样子如下：

In [2]:
train_dataframe.head()

,用户id,博文id,时刻,转发数量,评论数量,点赞数量,博文内容
0,d38e9bed5d98110dc2489d0d1cac3c2a,7d45833d9865727a88b960b0603c19f6,2015-02-23 17:41:29,0,0,0,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...
1,fa13974743d3fe6ff40d21b872325e9e,8169f1d45051e08ef213bf1106b1225d,2015-02-14 12:49:58,0,0,0,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...
2,da534fe87e7a52777bee5c30573ed5fd,68cd0258c31c2c525f94febea2d9523b,2015-03-31 13:58:06,0,0,0,淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了
3,e06a22b7e065e559a1f0bf7841a85c51,00b9f86b4915aedb7db943c54fd19d59,2015-06-11 20:39:57,0,4,3,看点不能说的，你们都懂[笑cry]
4,f9828598f9664d4e347ef2048ce17734,c7f6f66044c0c5a3330e2c5371be6824,2015-03-10 18:02:38,0,0,0,111多张


测试集的样子如下：

In [3]:
test_dataframe.head()

,用户id,博文id,时刻,博文内容
0,c01014739c046cd31d6f1b4fb71b440f,0cd5ef13eb11ed0070f7625b14136ec9,2015-08-19 22:44:55,Xah Emacs Tutorial http://t.cn/zWoY9IZ
1,fa5aed172c062c61e196eac61038a03b,7cce78a4ad39a91ec1f595bcc7fb5eba,2015-08-01 14:06:31,卖水果老人因没住处夜宿酒店门口 被车碾死 http://t.cn/RL0Hw8J （分享自@...
2,77fc723c196a45203e70f4d359c96946,a3494d8cf475a92739a2ffd421640ddf,2015-08-04 10:51:38,不要学习没有用的理论？ 不是：要学习，但要知道这个理论，为什么没有用？真实有用的理论是什么。...
3,e4097b07f34366399b623b94f174f60c,6b89aea5aa7af093dde0894156c49dd3,2015-08-16 14:59:19,[幸运之星] -恭喜！您的新浪微博账号已被系统确认为“新浪五周年”活动二等奖幸运用户。请登陆...
4,d43f7557c303b84070b13aa4eeeb21d3,0bdeff19392e15737775abab46dc5437,2015-08-04 22:30:46,【Lennart Poettering宣布首届Systemd会议】受争议Linux初始化系统...


In [4]:
# 将数据集拼接为总体数据集，统一进行特征工程处理，处理之后再分割为训练集和测试集
total_dataframe = pd.concat([train_dataframe, test_dataframe], axis=0)
total_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1407915 entries, 0 to 178296
Data columns (total 7 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   用户id    1407915 non-null  object
 1   博文id    1407915 non-null  object
 2   时刻      1407915 non-null  object
 3   转发数量    1229618 non-null  object
 4   评论数量    1229618 non-null  object
 5   点赞数量    1229618 non-null  object
 6   博文内容    1407915 non-null  object
dtypes: object(7)
memory usage: 85.9+ MB


目前来看，数据集中没有空值和异常值。暂不进行处理。

# 抽取格式化特征
### 抽取样本的日期特征

In [5]:
total_feature_df = pd.DataFrame()
feature_columns = []

def insert_feature_tag(feature_name, feature_type):
    feature_tag = (feature_name, feature_type)
    if feature_tag not in feature_columns:
        feature_columns.append(feature_tag)

import datetime
clock_seq = [datetime.datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in total_dataframe['时刻']]
clock_seq_month = [i.month for i in clock_seq]
clock_seq_weekday = [i.weekday() for i in clock_seq]
clock_seq_day = [i.day for i in clock_seq]
clock_seq_hour = [i.hour for i in clock_seq]
clock_seq_minute = [i.minute for i in clock_seq]
clock_seq_minute_index = [i*60+j for i,j in zip(clock_seq_hour, clock_seq_minute)]
total_feature_df['月计数'] = clock_seq_month
insert_feature_tag('月计数', 'i')
total_feature_df['周计数'] = clock_seq_weekday
insert_feature_tag('周计数', 'i')
total_feature_df['日计数'] = clock_seq_day
insert_feature_tag('日计数', 'i')
total_feature_df['时计数'] = clock_seq_hour
insert_feature_tag('时计数', 'i')
total_feature_df['分计数'] = clock_seq_minute
insert_feature_tag('分计数', 'i')
total_feature_df['日内总分钟数'] = clock_seq_minute_index
insert_feature_tag('日内总分钟数', 'f')

### 抽取微博文本中的话题标签，用户标签，表情标签，超链接

In [6]:
def type_2_condition(content):
    pattern_1 = re.compile(r'向.*?@.*?提问')
    pattern_2 = re.compile(r'我在.*?提问')
    type_2_label = 0
    kernel_word_list = ['分享', '发表', '评论@', '评论给', '刚刚下载了豆丁文档', '通过@微盘 下载了',
                        '我刚刚下载了', '我刚下载了', '我刚才下载了', '我在#茶图素材网#下载了', '我正在关注',
                        '我上传了', '赞了 @', '我参与了']

    if pattern_1.findall(content) or pattern_2.findall(content):
        type_2_label = 1
    elif len(content) < 101:
        for kernel_word in kernel_word_list:
            if kernel_word in content[:20]:
                type_2_label = 1
    return type_2_label

# 话题的结构模式
pattern_topic = re.compile(r'#(.+?)#')
# 用户的结构模式
pattern_user = re.compile(r'@(.+?)[ ，@]')
# 表情的结构模式
pattern_face = re.compile(r'\[(.+?)\]')
# 超链接的结构模式
pattern_url = re.compile(r'http://t.cn/[a-zA-Z0-9]{7}')
# 超链接在最末尾的结构模式(在正则表达式中$代表文本的结束)
pattern_url_at_tail = re.compile(r'http://t.cn/[a-zA-Z0-9]{7}$')
# 标题的结构模式
pattern_title = re.compile(r'【.{6,}?】')

content_seq = total_dataframe['博文内容'].values
topic_tag_list = []
topic_tag_num_list = []
user_tag_list = []
user_tag_num_list = []
face_tag_list = []
face_tag_num_list = []
url_tag_list = []
url_at_tail_list = []
pic_tag_list = []
from_other_tag_list = []
uncomplete_tag_list = []
video_tag_list = []
auto_gene_tag_list = []
new_tag_list = []
is_computer_tag_list = []
is_stock_tag_list = []
is_promotion_tag_list = []
valid_content_len_list = []
valid_content_list = []
for item_content in content_seq:
    # 话题标签
    topic_tag = pattern_topic.findall(item_content)
    topic_tag_list.append(';'.join(topic_tag))
    topic_tag_num_list.append(len(topic_tag))
    # 用户标签
    user_tag = pattern_user.findall(item_content)
    user_tag_list.append(';'.join(user_tag))
    user_tag_num_list.append(len(user_tag))
    # 表情标签
    face_tag = pattern_face.findall(item_content)
    face_tag_list.append(';'.join(face_tag))
    face_tag_num_list.append(len(face_tag))
    # 文本中超链接的数量
    url_tag = pattern_url.findall(item_content)
    url_tag_list.append(len(url_tag))
    # 文本中超链接是否被放置在最末尾
    if pattern_url_at_tail.search(item_content):
        url_at_tail_list.append(1)
    else:
        url_at_tail_list.append(0)
    # 是否分享了图片
    if '分享图片' in item_content:
        pic_tag_list.append(1)
    else:
        pic_tag_list.append(0)
    # 该博文是否为转发
    item_mid_from_other = int('//' in item_content and 'http:' not in item_content)
    from_other_tag_list.append(item_mid_from_other)
    # 该博文是否未完全显示（没有完全显示的博文会在末尾区域有...，并辅导一个链接或者点击查看等文字）
    # 这样的博文本身是很长的，有一定的特殊性，所以需要记录下来。这里的30是假定未完全显示的博文...应该出现
    # 在倒数30个字符以内的位置。
    if '...' in item_content[-30:]:
        uncomplete_tag_list.append(1)
    else:
        uncomplete_tag_list.append(0)
    # 该博文是否插入视频，经过观察，微博中插入视频的样本通常都有“视频：”这样的格式
    if '视频：' in item_content:
        video_tag_list.append(1)
    else:
        video_tag_list.append(0)
    # 该博文是否为软件自动生成
    if type_2_condition(item_content):
        auto_gene_tag_list.append(1)
    else:
        auto_gene_tag_list.append(0)
    # 该博文中是否有标题，博文中标题通常有【xxx】这样的结构，有标题的博文通常是新闻报道类的
    if pattern_title.search(item_content):
        new_tag_list.append(1)
    else:
        new_tag_list.append(0)
    # 计算去掉格式化信息之后的博文长度
    item_content_strip = re.sub(r'#.+?#', '', item_content)
    item_content_strip = re.sub(r'http://t.cn/[0-9a-zA-Z]{7}', '', item_content_strip)
    item_content_strip = re.sub(r'（.*?@.*?）$', '', item_content_strip.strip())
    item_content_strip = re.sub(r'@.+?[ ，]', '', item_content_strip)
    item_content_strip = re.sub(r'\[.+?\]', '', item_content_strip)
    item_content_strip = re.sub(r'\(.*?\)$', '', item_content_strip)
    item_content_strip = re.sub(r'（.*?）$', '', item_content_strip)
    item_content_strip = re.sub(r'@.*?$', '', item_content_strip)
    valid_content_len_list.append(len(item_content_strip))
    valid_content_list.append(item_content_strip)
    # 对训练集的观察发现，与计算机相关的博文往往互动行为较多，所以可以通过一些关键词来判断该博文是否与计算机相关
    item_content_is_computer = 0 
    for keyword in ['Python', 'python', 'Java', 'java', 'JAVA', '计算机', '程序', '漏洞', '病毒',
                    '智能', '甲骨文', '谷歌', '微软', 'TensorFlow', 'PyTorch', 'Web', 'web', '云技术', '物联网',
                    'IT', '后端', '前端', '客户端', '科普', '硬件', '软件', '云计算', '开源', 'Git',
                    '代码', '框架', '算法', '版本', '科学', '数据挖掘', '数据库', '源码', 'bug', '数据分析', '服务器',
                   '互联网', 'Linux', '跨平台', 'CSS', 'SQL', '树莓派', '集群']:
        if keyword in item_content_strip:
            item_content_is_computer = 1
            break
    is_computer_tag_list.append(item_content_is_computer)
    # 微博中有大量荐股或者投资广告，所以这也是一类重要的博文，需要单独识别出来，我们也设定了一组关键词
    item_content_is_stock = 0
    for keyword in ['股票', '炒股', '理财', '投资', '盈利', '本金', '上证', '深证', '交易', '牛股', '指数',
                    '纳斯达克', '开盘', '收盘', '收益率', '股市', '套牢', '股神', '财经',
                    '宏观', '微观', '经济', '财报', '上市', '公司', '坐庄', '暴跌', '下跌', '股民']:
        if keyword in item_content_strip:
            item_content_is_stock = 1
            break
    is_stock_tag_list.append(item_content_is_stock)
    # 微博中有大量荐股或者投资广告，所以这也是一类重要的博文，需要单独识别出来，我们也设定了一组关键词
    item_content_is_promotion = 0
    for keyword in ['红包', '转发', '推广', '点赞', '评论', '免费', '现金', '收藏', '广告', '签到', '福利', 'QQ群',
                    '专享', '小财神', '手气', '礼包', '券', '好运', '速来', '抽奖', '下载',
                     '查收', '领取', '打车']:
        if keyword in item_content_strip:
            item_content_is_promotion = 1
            break
    is_promotion_tag_list.append(item_content_is_promotion)

column_name = '话题标签列表'
column_type = 'i'
total_feature_df[column_name] = topic_tag_list
insert_feature_tag(column_name, column_type)
column_name = '话题标签数量'
column_type = 'f'
total_feature_df[column_name] = topic_tag_num_list
insert_feature_tag(column_name, column_type)
column_name = '用户标签列表'
column_type = 'i'
total_feature_df[column_name] = user_tag_list
insert_feature_tag(column_name, column_type)
column_name = '用户标签数量'
column_type = 'f'
total_feature_df[column_name] = user_tag_num_list
insert_feature_tag(column_name, column_type)
column_name = '表情标签列表'
column_type = 'i'
total_feature_df[column_name] = face_tag_list
insert_feature_tag(column_name, column_type)
column_name = '表情标签数量'
column_type = 'f'
total_feature_df[column_name] = face_tag_num_list
insert_feature_tag(column_name, column_type)
column_name = '超链接标签数量'
column_type = 'f'
total_feature_df[column_name] = url_tag_list
insert_feature_tag(column_name, column_type)
column_name = '尾部超链接标记'
column_type = 'i'
total_feature_df[column_name] = url_at_tail_list
insert_feature_tag(column_name, column_type)
column_name = '图片标记'
column_type = 'i'
total_feature_df[column_name] = pic_tag_list
insert_feature_tag(column_name, column_type)
column_name = '转发标记'
column_type = 'i'
total_feature_df[column_name] = from_other_tag_list
insert_feature_tag(column_name, column_type)
column_name = '未完全显示标记'
column_type = 'i'
total_feature_df[column_name] = uncomplete_tag_list
insert_feature_tag(column_name, column_type)
column_name = '视频标记'
column_type = 'i'
total_feature_df[column_name] = video_tag_list
insert_feature_tag(column_name, column_type)
column_name = '自动生成标记'
column_type = 'i'
total_feature_df[column_name] = auto_gene_tag_list
insert_feature_tag(column_name, column_type)
column_name = '新闻标记'
column_type = 'i'
total_feature_df[column_name] = new_tag_list
insert_feature_tag(column_name, column_type)
column_name = '计算机标记'
column_type = 'i'
total_feature_df[column_name] = is_computer_tag_list
insert_feature_tag(column_name, column_type)
column_name = '股票标记'
column_type = 'i'
total_feature_df[column_name] = is_stock_tag_list
insert_feature_tag(column_name, column_type)
column_name = '推广标记'
column_type = 'i'
total_feature_df[column_name] = is_promotion_tag_list
insert_feature_tag(column_name, column_type)
column_name = '实际博文长度'
column_type = 'f'
total_feature_df[column_name] = valid_content_len_list
insert_feature_tag(column_name, column_type)
column_name = '截断博文内容'
column_type = 'i'
total_feature_df[column_name] = valid_content_list
insert_feature_tag(column_name, column_type)

### 用户属性特征
上面的工作完成了对每一条博文的文本特征抽取，而由于社交网络的存在，同样的博文由不同的用户发布所能够获得的互动行为是迥然不同的。所以需要统计出不同用户的特征以辅助对每一条博文进行预测。

In [20]:
total_feature_df['uid'] = total_dataframe['uid']
uid_feature_df = total_feature_df.groupby('uid')
for uid, uid_df in uid_feature_df:
    pass

0.8820255789735907